#### Mobile Net

In [6]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import preprocess_input 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from keras.applications.mobilenet import MobileNet
from sklearn.metrics import confusion_matrix, classification_report

In [7]:
#Re-size image 
image_size = [224, 224]

In [8]:
train_path = 'Dataset/train'
test_path = 'Dataset/test'

In [10]:
vgg = MobileNet(input_shape=image_size + [3], include_top = False)
#Cutting the last layer

17235968/17225924 [==============================] - 78s 5us/step


In [11]:
#Don't need train this model
for layer in vgg.layers:
    layer.trainable = False

In [12]:
folders = glob('/Users/vignaya/Documents/python/Machine_Learning_A-Z_(Codes_and_Datasets)/Part 8 - Deep Learning/Section 40 - Convolutional Neural Networks (CNN)/Python/Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set/*')

In [13]:
x = Flatten()(vgg.output)
predictions = Dense(1, activation = 'sigmoid')(x)

In [14]:
model = Model(inputs = vgg.input, outputs = predictions)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [15]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

##### Ignore this

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(
                        '/Users/vignaya/Documents/python/Machine_Learning_A-Z_(Codes_and_Datasets)/Part 8 - Deep Learning/Section 40 - Convolutional Neural Networks (CNN)/Python/Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set/',
                                                 target_size = image_size,
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [17]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
    '/Users/vignaya/Documents/python/Machine_Learning_A-Z_(Codes_and_Datasets)/Part 8 - Deep Learning/Section 40 - Convolutional Neural Networks (CNN)/Python/Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set/',
                                            target_size = image_size,
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [18]:
test_set.image_shape

(224, 224, 3)

In [21]:
model.fit(training_set, epochs = 2, steps_per_epoch=len(training_set), validation_data=test_set, validation_steps=len(test_set))

Epoch 1/2
250/250 [==============================] - 523s 2s/step - loss: 0.3656 - accuracy: 0.9300 - val_loss: 0.0934 - val_accuracy: 0.9876
Epoch 2/2
250/250 [==============================] - 492s 2s/step - loss: 0.1569 - accuracy: 0.9812 - val_loss: 0.1149 - val_accuracy: 0.9865


In [22]:
pd.DataFrame(model.history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.270112,0.958250,0.093429,0.987625
1,0.154191,0.982375,0.114914,0.986500


In [23]:
pred = model.predict(test_set)

In [25]:
pred_val = pred>0.5

In [31]:
pred_train = model.predict(training_set)
pred_train_val = pred_train>0.5

In [26]:
test_set.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [28]:
confusion_matrix(test_set.classes, pred_val)

array([[1993, 2007],
       [1919, 2081]])

In [30]:
print(classification_report(test_set.classes, pred_val))

              precision    recall  f1-score   support

           0       0.51      0.50      0.50      4000
           1       0.51      0.52      0.51      4000

    accuracy                           0.51      8000
   macro avg       0.51      0.51      0.51      8000
weighted avg       0.51      0.51      0.51      8000

